In [1]:
import sys
sys.path.append("./work")

from utils.configuration import get_settings
from operators.streaming import SparkStreaming
import pyspark.sql.functions as F

# Create Spark Session by user defined class

In [2]:
settings = get_settings()
spark = SparkStreaming.get_instance(app_name="Spark Streaming")

In [21]:
spark

# Check the Spark MongoDB Connector

In [3]:
df = spark.read.format("mongodb") \
            .option("spark.mongodb.read.connection.uri", settings.MONGO_ATLAS_PYTHON) \
            .option("spark.mongodb.read.database", "imcp") \
            .option("spark.mongodb.read.collection", "raw") \
            .load()

df = df.withColumn("created_time", F.to_timestamp(F.col("created_time"), "yyyy-MM-dd HH:mm:ss")) \
        .withColumn("caption_size", F.size(F.split(df["caption"], " ")))

print(df.printSchema())
print("The number of filterd rows: ", df.filter(df["caption_size"] < 100).count())

root
 |-- _id: string (nullable = true)
 |-- caption: string (nullable = true)
 |-- created_time: timestamp (nullable = true)
 |-- howpublished: string (nullable = true)
 |-- publisher: string (nullable = true)
 |-- short_caption: string (nullable = true)
 |-- url: string (nullable = true)
 |-- caption_size: integer (nullable = false)

None
The number of filterd rows:  187758


In [4]:
spark.stop()